<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">TRANSFORMACIÓN DEL CONJUNTO DE DATOS PARA EL DESARROLLO DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [1]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing/data')

Mounted at /content/drive


In [2]:
import pandas as pd # Manipulación y el analísis de datos.
import sqlite3 as sql # Interactuar con base de datos SQL usando Python.
from sklearn.preprocessing import MultiLabelBinarizer #Transformación entre iterables de iterables y un formato multietiqueta.
import shutil # Ayuda a automatizar el proceso de copia y eliminación de archivos y directorios.

#### CONEXIÓN CON LA BASE DE DATOS

In [64]:
#Copia de la base de datos original
db_movies = '/content/drive/MyDrive/marketing/Marketing/data/db_movies'
db_movies_copy = '/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db'
shutil.copy(db_movies, db_movies_copy)

'/content/drive/MyDrive/marketing/Marketing/data/db_movies_copy.db'

In [65]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect(db_movies_copy)

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [66]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[('ratings',), ('movies',)]

#### EXPLORACIÓN DE LAS TABLAS

##### TABLA MOVIES

In [77]:
# Ejecuta la consulta SQL "SELECT * FROM movies" sobre la base de datos y carga los resultados en un DataFrame de pandas.
table_movies = pd. read_sql_query("SELECT * FROM movies", conexion)
table_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy





```
# Transformaciones requeridas en la tabla movies
  - Separación del nombre de la película y el año de estreno.
  - Separación de la columna de género en columnas binarias.
  Columna genres contiene múltiples géneros por cada fila.
```





In [78]:
# Uso de expresiones regulares de pandas para separa el nombre de la película del año.
table_movies['año'] = table_movies['title'].str.extract(r'\((\d{4})\)', expand=False) # Separación al año estreno, de lo contrario Nan.
table_movies['titulo'] = table_movies['title'].str.replace(r'\s?\(\d{4}\)', '', regex=True) #Separación del nombre de la película
table_movies.drop('title', axis=1, inplace=True) # Eliminación de la columna que contiene todos los géneros.
table_movies.head()

,movieId,genres,año,titulo
0,1,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story
1,2,Adventure|Children|Fantasy,1995,Jumanji
2,3,Comedy|Romance,1995,Grumpier Old Men
3,4,Comedy|Drama|Romance,1995,Waiting to Exhale
4,5,Comedy,1995,Father of the Bride Part II




```
# Debido a que los nombres de las películas y la posición del año de estreno
no tiene un formato uniforme, al intentar construir la separación con SQLite
estuvo muy procesa a generar errores y eliminar registros.

Las expresiones regulares de Pandas tiene mejores resultados para el
procesamiento de cadenas de texto complejas.
```



In [79]:
# Separar los géneros en listas
table_movies['genres'] = table_movies['genres'].str.split('|')

# Binarizar los géneros
mlb = MultiLabelBinarizer()
generos_bin = mlb.fit_transform(table_movies['genres'])

# Crear DataFrame con los géneros binarizados
df_generos = pd.DataFrame(generos_bin, columns=mlb.classes_, index=table_movies.index)


# Unir los géneros binarizados al DataFrame original
table_movies = pd.concat([table_movies, df_generos], axis=1)

#Eliminar la columna genres
table_movies.drop('genres', axis=1, inplace=True)
table_movies.drop('(no genres listed)', axis=1, inplace=True) #La tabla separa un pequeño grupo de que no tiene genero.

# Mostrar las primeras filas del resultado
table_movies.head()

,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,1995,Toy Story,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1995,Jumanji,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,1995,Grumpier Old Men,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,1995,Waiting to Exhale,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,1995,Father of the Bride Part II,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0




```
# En la separación de los géneros, una columna correspondió a aquellas
películas que no estaban identificadas con un género.
La columna se elimina y se busca explorar esos registros.
```



In [80]:
# Guardar el DataFrame con las transformaciones en la base de datos de la tabla movies.
table_movies.to_sql('movies', conexion, if_exists='replace', index=False)

#Ayuda a guardar los cambios
conexion.commit()

In [84]:
#Películas que registran sin ningún tipo de genero asignado.
peliculas_sin_genero = table_movies[
    table_movies.drop(['movieId', 'año', 'titulo'], axis=1).sum(axis=1) == 0][['movieId', 'titulo', 'año']]

peliculas_sin_genero

,movieId,titulo,año
8517,114335,La cravate,1957
8684,122888,Ben-hur,2016
8687,122896,Pirates of the Caribbean: Dead Men Tell No Tales,2017
8782,129250,Superfast!,2015
8836,132084,Let It Be Me,1995
8902,134861,Trevor Noah: African American,2013
9033,141131,Guardians,2016
9053,141866,Green Room,2015
9070,142456,The Brand New Testament,2015
9091,143410,Hyena Road,NaN




```
# En total tenemos 34 registros de películas que no tienen asignado ningún género.
Se determino realizar la elimación de estos registros.
```



In [85]:
#Eliminación de los registros sin genero.
table_movies.drop(table_movies[table_movies.drop(['movieId', 'año', 'titulo'], axis=1).sum(axis=1) == 0].index, inplace=True)

In [91]:
#Valores nulos de de la tabla movies transformada.
table_movies.isnull().sum()

,0
movieId,0
año,4
titulo,0
Action,0
Adventure,0
Animation,0
Children,0
Comedy,0
Crime,0
Documentary,0


In [92]:
# Mostrar las películas que no tienen año registrado
peliculas_sin_anio = table_movies[table_movies['año'].isna()]
display(peliculas_sin_anio)

,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
6059,40697,NaN,Babylon 5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
9031,140956,NaN,Ready Player One,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
9179,149334,NaN,Nocturnal Animals,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9367,162414,NaN,Moonlight,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0




```
# Se identificaron 4 registros de películas que no tiene el año
de estreno, se tiene dos opciones:
  -Eliminar los registros.
  -Completarlos con el año de estreno buscando la información,
  considerando que son pocos registros.
```



In [95]:
year_mapping = {
    "Babylon 5": 1994,
    "Ready Player One": 2018,
    "Nocturnal Animals":2016,
    "Moonlight":2016

}

for title, year in year_mapping.items():
    table_movies.loc[table_movies['titulo'] == title, 'año'] = year

# Now, let's handle the remaining missing years, for example by removing them
table_movies.dropna(subset=['año'], inplace=True)

# Convert the 'año' column to integer type
table_movies['año'] = table_movies['año'].astype(int)

# Save the updated DataFrame back to the database
table_movies.to_sql('movies', conexion, if_exists='replace', index=False)
conexion.commit()


In [98]:
# Filtrar las filas donde el título está duplicado
titulos_duplicados = table_movies[table_movies['titulo'].duplicated(keep=False)]
display(titulos_duplicados.sort_values('titulo'))


,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
7332,77846,1997,12 Angry Men,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
905,1203,1957,12 Angry Men,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9051,141844,1971,12 Chairs,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9046,141816,1976,12 Chairs,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
777,1019,1954,"20,000 Leagues Under the Sea",0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5473,26198,1968,"Yours, Mine and Ours",0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9258,156553,2015,Zoom,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6266,47384,2006,Zoom,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4114,5899,1964,Zulu,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0




```
# Al verificar la existencia de registros duplicados,
un grupo de películas se repiten con el nombre, pero el año de estreno
y id de la película es diferente, dando a entender que es un remake.
```



In [99]:
duplicados_titulo_anio = table_movies[table_movies.duplicated(subset=['titulo', 'año'], keep=False)]
display(duplicados_titulo_anio.sort_values(['titulo', 'año']))


,movieId,año,titulo,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
4169,6003,2002,Confessions of a Dangerous Mind,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,1,0,0
9106,144606,2002,Confessions of a Dangerous Mind,0,0,0,0,1,1,0,...,0,0,0,0,0,1,0,1,0,0
650,838,1996,Emma,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
5601,26958,1996,Emma,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
5854,32600,2004,Eros,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9135,147002,2004,Eros,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2141,2851,1980,Saturn 3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
9468,168358,1980,Saturn 3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
5931,34048,2005,War of the Worlds,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
6932,64997,2005,War of the Worlds,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


##### TABLA RATINGS

In [ ]:
# Ejecuta la consulta SQL "SELECT * FROM ratings" sobre la base de datos y carga los resultados en un DataFrame de pandas.
table_ratings = pd.read_sql_query("SELECT * FROM ratings", conexion)
table_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352




```
# Transformaciones requeridas para la tabla rating
 - La columna timestamp almacena información de fecha y hora,
 incluyendo el año, mes, día, hora, minuto y segundo, se puede separar
 para hacer consultas.
```



In [ ]:
#Transformación de la columna timestampo por fecha y hora.
table_ratings['timestamp'] = pd.to_datetime(table_ratings['timestamp'], unit='s')
table_ratings['day'] = table_ratings['timestamp'].dt.day
table_ratings['month'] = table_ratings['timestamp'].dt.month

table_ratings['hour'] = table_ratings['timestamp'].dt.hour
table_ratings['year'] = table_ratings['timestamp'].dt.year

if 'date' in table_ratings.columns:
  table_ratings = table_ratings.drop('date', axis=1)

#Eliminar la columna timestampo
table_ratings.drop(columns=['timestamp'], inplace=True)

# Mostrar el DataFrame actualizado
table_ratings


,userId,movieId,rating,day,month,hour,year
0,1,1,4.0,30,7,18,2000
1,1,3,4.0,30,7,18,2000
2,1,6,4.0,30,7,18,2000
3,1,47,5.0,30,7,19,2000
4,1,50,5.0,30,7,18,2000
...,...,...,...,...,...,...,...
100831,610,166534,4.0,3,5,21,2017
100832,610,168248,5.0,3,5,22,2017
100833,610,168250,5.0,8,5,19,2017
100834,610,168252,5.0,3,5,21,2017


In [ ]:
# Filtrar ratings para que solo incluyan movieId que están en table_movies
table_ratings = table_ratings[table_ratings['movieId'].isin(table_movies['movieId'])]
table_ratings.shape

(100781, 7)

In [ ]:
# Guardar el DataFrame con las transformaciones en la base de datos de la tabla ratings.
table_ratings.to_sql('ratings', conexion, if_exists='replace', index=False)

#Ayuda a guardar los cambios
conexion.commit()

In [ ]:
#Cierre de la conexión con la base de datos.
conexion.close()